#  Librerías

### Librerías pesadas
Para ejecutar solo una vez

In [1]:
import math
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np
import tqdm
import wandb

/mnt/shared/Documents/proyecto-recsys/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!wandb login 99217068fbd71985701543b0c0064e805ac87449

wandb: Appending key for api.wandb.ai to your netrc file: /home/rafael/.netrc


### Librerías livianas
Para ejecutar múltiples veces

# Configuración General

Variables relacionadas al procesamiento de datos y del modelo en sí

### Variables de Preprocesamiento

In [3]:

# Porcentaje para usar solo una fracción del dataset de usuario.
# si al eliminar usuarios quedan viajes o POI sin visitas, estos también
# serán eliminados
USER_FRAC = 0.2
MIN_POI_VISITS = 5
MAX_SEQUENCES_PER_USER = 100
SEQUENCE_LENGTH = 14

In [4]:
BATCH_SIZE=64
EPOCHS=100

In [5]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 80

# Gowalla Dataset

In [6]:
! ./download-gowalla.sh

zsh:1: no such file or directory: ./download-gowalla.sh


In [7]:
# !mkdir -p download
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=FILEID' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=0BzpKyxX1dqTYRTFVYTd1UG81ZXc" -O download/gowalla.zip && rm -rf /tmp/cookies.txt
# !unzip download/gowalla.zip -d download

### Cargar Datos

In [8]:
users    = pd.read_csv('download/gowalla/gowalla_userinfo.csv')
friends  = pd.read_csv('download/gowalla/gowalla_friendship.csv')
checkins = pd.read_csv('download/gowalla/gowalla_checkins.csv')
pois_1   = pd.read_csv('download/gowalla/gowalla_spots_subset1.csv', encoding='iso-8859-1')
pois_2   = pd.read_csv('download/gowalla/gowalla_spots_subset2.csv', encoding='iso-8859-1')
pois     = pd.concat((pois_1, pois_2), ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'download/gowalla/gowalla_userinfo.csv'

# Preprocesamiento

### Usuarios

Revisamos la distrubución de checkins de usuarios

In [ ]:
users.sample(5)

,id,bookmarked_spots_count,challenge_pin_count,country_pin_count,highlights_count,items_count,photos_count,pins_count,province_pin_count,region_pin_count,state_pin_count,trips_count,friends_count,stamps_count,checkin_num,places_num
219885,1906922,0,1,1,0,1,0,3,0,2,1,0,2,1,2,1
306158,2269155,0,1,0,0,0,0,1,0,0,0,0,2,0,0,0
1092,2281,0,7,1,3,4,5,11,0,4,3,0,11,49,70,49
202077,1426773,0,1,0,0,0,0,1,0,0,0,0,2,0,0,0
110417,240065,0,2,1,0,2,0,4,0,2,1,0,1,4,4,4


In [ ]:
users.checkin_num.describe()

count    407533.000000
mean         88.341212
std         435.982581
min           0.000000
25%           1.000000
50%          10.000000
75%          52.000000
max       46981.000000
Name: checkin_num, dtype: float64

nos quedamos con un porcentaje de los usuarios y los filtramos los usuarios segun los checkins que tengan

In [ ]:
users.checkin_num.describe()

count    407533.000000
mean         88.341212
std         435.982581
min           0.000000
25%           1.000000
50%          10.000000
75%          52.000000
max       46981.000000
Name: checkin_num, dtype: float64

In [ ]:
print('Current users', len(users))
users = users.sample(frac=USER_FRAC)
users = users[(users.checkin_num >= users.checkin_num.quantile(0.1)) &
              (users.checkin_num <= users.checkin_num.quantile(0.9)) &
              (users.checkin_num >= SEQUENCE_LENGTH)]
users = users[['id']]
print('Reduced users', len(users))

Current users 407533
Reduced users 28672


### Amigos

In [ ]:
friends.sample(5)

,userid1,userid2
3825106,1742754,1723314
225023,2106567,2379930
715219,1094516,213106
2461113,2357114,2142435
2741226,309765,1059248


### Checkins

In [ ]:
checkins.sample(5)

,userid,placeid,datetime
16394571,136105,664278,2010-03-28T17:42:23Z
22382659,56311,21924,2010-06-11T18:32:34Z
24015061,191424,1140938,2010-05-31T01:37:36Z
29355460,384874,216417,2010-09-27T08:55:44Z
19255341,95358,18415,2010-03-31T22:19:14Z


Eliminamos los checkins de los usuarios no sampleados

In [ ]:
print('Current checkins', len(checkins))
checkins = pd.merge(checkins, users, how='inner', left_on='userid', right_on='id', copy=False)[checkins.columns]
checkins = checkins.reset_index(drop=True)
print('Reduced checkins', len(checkins))

Current checkins 36001959
Reduced checkins 1686233


### POIS

In [ ]:
pois.sample(5)

,id,created_at,lng,lat,photos_count,checkins_count,users_count,radius_meters,highlights_count,items_count,max_items_count,spot_categories,name,city_state,Unnamed: 5,Unnamed: 6
2206230,6873526,2011-02-05T22:00:49Z,-4.315520,55.888856,0.0,4.0,1.0,75.0,0.0,0.0,10.0,"[{'url': '/categories/412', 'name': 'Fish & Ch...",NaN,NaN,NaN,NaN
2125023,6772143,2011-01-17T17:05:48Z,8.966019,48.625504,2.0,4.0,4.0,75.0,0.0,0.0,10.0,"[{'url': '/categories/69', 'name': 'Mission'}]",NaN,NaN,NaN,NaN
1254913,1438155,2010-07-14T13:09:45Z,14.746399,49.106804,0.0,12.0,3.0,250.0,0.0,1.0,10.0,"[{'url': '/categories/114', 'name': 'Lake & Po...",NaN,NaN,NaN,NaN
2176901,6836698,2011-01-29T18:51:02Z,-79.385299,43.768896,0.0,3.0,2.0,75.0,0.0,0.0,10.0,"[{'url': '/categories/18', 'name': 'Asian Food'}]",NaN,NaN,NaN,NaN
1129418,1292122,2010-06-17T02:10:28Z,10.709860,60.772538,4.0,44.0,3.0,35.0,0.0,1.0,10.0,"[{'url': '/categories/87', 'name': 'Duplex'}]",NaN,NaN,NaN,NaN


Filtramos los pois si han sido visitadas pocas veces según los parámetros que definimos

In [ ]:
visited_pois = pd.merge(pois, checkins, left_on='id', right_on='placeid', how='inner', copy=False)

In [ ]:
visited_pois['visited_count'] = np.zeros(len(visited_pois))

visited_pois = visited_pois[['id', 'visited_count']].groupby(by='id').count()
visited_pois = visited_pois[visited_pois.visited_count >= MIN_POI_VISITS]

pois = pd.merge(pois, visited_pois, on='id', how='inner', copy=False)

Nos quedamos con sólo las columnas que nos importan

In [ ]:
pois = pois[['id', 'lat', 'lng', 'visited_count']]
pois 

,id,lat,lng,visited_count
0,8947,37.331880,-122.029631,83
1,8961,32.940099,-97.115691,8
2,8964,39.041053,-94.594739,9
3,8972,39.057217,-94.605586,6
4,8976,39.051633,-94.590633,5
...,...,...,...,...
73179,7482495,46.661030,7.731774,5
73180,7507262,13.730346,100.594728,6
73181,7509935,51.916201,8.414108,6
73182,7514781,26.092261,43.973677,6


Ahora eliminamos los checkins de pois que ya no existen

In [ ]:
print('Current checkins', len(checkins))
checkins = pd.merge(pois, checkins, left_on='id', right_on='placeid', how='inner', copy=False)[checkins.columns]
checkins = checkins.reset_index(drop=True)
print('Reduced checkins', len(checkins))

Current checkins 1686233
Reduced checkins 917160


Finalmente eliminamos nuevamente a los usuarios que se quedaron sin suficientes checkins

In [ ]:
print('Current users', len(users))
users = pd.merge(checkins, users, how='inner', left_on='userid', right_on='id', copy=False)[users.columns].drop_duplicates()
print('Reduced users', len(users))

Current users 28672
Reduced users 28296


### Reasignación de IDs

In [ ]:
users = users.reset_index(drop=True)
users['user_sid'] = users.index

pois = pois.reset_index(drop=True)
pois['place_sid'] = pois.index

### Agregar Datos

Crearemos un dataset unificado que usaremos para entrenar el modelo de los embeddings

In [ ]:
pois.head()

,id,lat,lng,visited_count,place_sid
0,8947,37.331880,-122.029631,83,0
1,8961,32.940099,-97.115691,8,1
2,8964,39.041053,-94.594739,9,2
3,8972,39.057217,-94.605586,6,3
4,8976,39.051633,-94.590633,5,4


In [ ]:
pois.sort_values(by='visited_count', ascending=False)

,id,lat,lng,visited_count,place_sid
2856,23519,13.689897,100.748320,1942,2856
17369,155746,13.746659,100.534912,1671,17369
7373,55033,59.330158,18.058079,1617,7373
8624,66171,60.193511,11.098251,1524,8624
7721,58725,59.650051,17.932262,1512,7721
...,...,...,...,...,...
24882,313406,36.112865,-115.170482,5,24882
55367,4190354,4.900707,114.927538,5,55367
24888,313534,34.074817,-118.377514,5,24888
24889,313546,42.347065,-71.096789,5,24889


In [ ]:
users.head()

,id,user_sid
0,20547,0
1,61109,1
2,43056,2
3,1979414,3
4,2336690,4


In [ ]:
checkins.head()

,userid,placeid,datetime
0,20547,8947,2010-04-01T22:26:23Z
1,61109,8947,2011-05-16T17:06:59Z
2,43056,8947,2010-01-22T01:46:01Z
3,1979414,8947,2010-11-19T00:27:14Z
4,1979414,8947,2010-11-19T00:27:06Z


In [ ]:
users_checkins = pd.merge(users, checkins, left_on='id', right_on='userid', copy=False).drop('id', axis=1)
users_checkins = pd.merge(users_checkins, pois[['id', 'place_sid']], left_on='placeid', right_on='id', copy=False)
users_checkins['date'] = pd.to_datetime(users_checkins['datetime'])
users_checkins = users_checkins.drop('datetime', axis=1)
users_checkins.sort_values(by=['user_sid', 'date'], inplace=True)
users_checkins.tail()

,user_sid,userid,placeid,id,place_sid,date
917156,28295,736841,7389396,7389396,73152,2011-05-23 19:59:36+00:00
917155,28295,736841,7389396,7389396,73152,2011-05-24 11:16:00+00:00
917154,28295,736841,7389396,7389396,73152,2011-05-30 12:19:54+00:00
917153,28295,736841,7389396,7389396,73152,2011-06-02 11:29:26+00:00
917152,28295,736841,7389396,7389396,73152,2011-06-09 12:19:36+00:00


Podemos ver que hay un problema: Muchas veces se repiten los POI consecutivos de un usuario, sin embargo eso no aporta mucha información al embedding, por lo que los eliminaremos en el siguiente paso

In [ ]:
users_checkins['last_place_sid'] = users_checkins['place_sid'].shift(1)
users_checkins = users_checkins[users_checkins.place_sid != users_checkins.last_place_sid]
users_checkins = users_checkins.drop('last_place_sid', axis=1)

In [ ]:
users_checkins.tail()

,user_sid,userid,placeid,id,place_sid,date
917108,28292,2434347,7109931,7109931,73073,2011-04-12 13:05:49+00:00
917127,28292,2434347,7111148,7111148,73075,2011-04-13 00:18:49+00:00
917146,28293,2494754,7286307,7286307,73125,2011-04-25 10:30:09+00:00
917151,28294,1737255,7290911,7290911,73127,2011-04-26 11:56:35+00:00
917159,28295,736841,7389396,7389396,73152,2011-05-17 15:02:18+00:00


In [ ]:
from collections import defaultdict

In [ ]:
user_poi_seq = defaultdict(lambda: [])
user_date_seq = defaultdict(lambda: [])

In [ ]:
for user_sid, place_sid, date in zip(users_checkins['user_sid'], users_checkins['place_sid'], users_checkins['date']):
    user_poi_seq[user_sid].append(place_sid)
    user_date_seq[user_sid].append(date)

Generamos secuencias de puntos de interes visitados por usuarios de un largo predefinido para entrenar el modelo de embeddings

In [ ]:
from random import sample

poi_sequence_dataset = []

for user_sid, sequence in user_poi_seq.items():
    if len(sequence) < SEQUENCE_LENGTH: continue

    candidate_indexes = list(range(0, len(sequence) - SEQUENCE_LENGTH, SEQUENCE_LENGTH))

    n_sequences = min(len(candidate_indexes), MAX_SEQUENCES_PER_USER)
    start_indexes = sample(candidate_indexes, n_sequences)

    for idx in start_indexes:
        new_seq = sequence[idx:idx + SEQUENCE_LENGTH]
        poi_sequence_dataset.append(new_seq) 
    

In [ ]:
unique_pois = { poi for sequence in  poi_sequence_dataset  for poi in sequence }

In [ ]:
# 

# rands = np.random.rand(len(poi_sequence_dataset))

# total = 0
# for idx, seq in enumerate(poi_sequence_dataset):
#   if idx % 100000 == 0:
#     print(idx, '/', len(poi_sequence_dataset))

#   index = int(rands[idx] *  len(pois_list))
#   total += 33991 in seq[-5:]

# print(total / len(poi_sequence_dataset))
  

In [ ]:
def split_list(input, frac=0.5):
    split_index = int(len(input) * frac)
    return input[:split_index], input[split_index:]

## Split de Datos

Realizamos separacion en train / test / split de 80 / 10 / 10

In [ ]:
train_poi_sequence, rest = split_list(poi_sequence_dataset, 0.8)
test_poi_sequence, val_poi_sequence = split_list(rest)

In [ ]:
def split_history_target(sequences):
    history = [ seq[:-1] for seq in sequences]
    targets = [ seq[-1] for seq in sequences]
    return history, targets

In [ ]:
train_seq_history, train_seq_target = split_history_target(train_poi_sequence)
test_seq_history, test_seq_target = split_history_target(test_poi_sequence)
val_seq_history, val_seq_target = split_history_target(val_poi_sequence)

In [ ]:
unique_pois = { poi for sequence in  poi_sequence_dataset  for poi in sequence }

print("Total pois incluidos en dataset:", len(unique_pois))
print("Porcentaje de POIs que se usaran en el modelo",  len(unique_pois) * 100 / len(pois))

Total pois incluidos en dataset: 67824
Porcentaje de POIs que se usaran en el modelo 92.67599475295147


In [ ]:
print("Total Train", len(train_seq_history))
print("Total Test ", len(test_seq_history))
print("Total Val  ", len(val_seq_history))

Total Train 32872
Total Test  4109
Total Val   4109


Ya tenemos nuestros datos listos para entrenar!

# Modelo

In [ ]:
class EmbeddingModel(nn.Module):
    def __init__(self, vocab_size=None, emb_dim=None, hidden_dim=None, sample_length=None):
        super(EmbeddingModel, self).__init__()

        print("AAA", vocab_size, emb_dim, hidden_dim, sample_length)
        
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.hidden = nn.Linear(sample_length * emb_dim, hidden_dim)
        self.hidden_activation = nn.ReLU()
        self.output = nn.Linear(hidden_dim, vocab_size)
        self.output_activation = nn.LogSoftmax(dim=-1)

    def forward(self, xs):
        batch_size = xs.size()[0]

        # embed and merge
        xs = self.emb(xs)
        xs = torch.reshape(xs, (batch_size, -1))

        # hidden layer
        hidden = self.hidden(xs)
        hidden = self.hidden_activation(hidden)

        # output log probabilities
        output_logits = self.output(hidden)
        output_log_probs = self.output_activation(output_logits)
        
        return output_log_probs

    def predict(self, xs):
        return torch.argmax(self.forward(xs))

# Entrenamiento

In [ ]:
from torch.utils.data import DataLoader

def dataset_to_tensors(sequences, targets):
    return [[torch.tensor(x), torch.tensor(y)] for x, y in zip(sequences, targets)]


train_tensors = dataset_to_tensors(train_seq_history, train_seq_target)
train_dataloader = DataLoader(train_tensors, batch_size=BATCH_SIZE, shuffle=True)

test_tensors = dataset_to_tensors(test_seq_history, test_seq_target)
test_dataloader = DataLoader(test_tensors, batch_size=BATCH_SIZE, shuffle=True)

val_tensors = dataset_to_tensors(val_seq_history, val_seq_target)
val_dataloader = DataLoader(val_tensors, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
class EarlyStopper:
    def __init__(self, patience=5, min_delta_percent=1.02):
        self.patience = patience
        self.min_delta_percent = min_delta_percent
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > self.min_validation_loss * self.min_delta_percent:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

class Trainer:
    BASE_LR = 1e-2
    EPOCHS = 10
    PRINT_EVERY = 100
    VAL_EVERY = 500

    def __init__(self, vocab_size, embedding_dim, hidden_dim, sequence_length, train_dataloader, val_dataloader):
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.sequence_length = sequence_length
        self.model = EmbeddingModel(
            vocab_size=self.vocab_size, emb_dim=self.embedding_dim, 
            hidden_dim=self.hidden_dim, sample_length=self.sequence_length
        )

        self.loss = nn.NLLLoss()
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.BASE_LR) 
        # self.optimizer = optim.Adam(self.model.parameters(), amsgrad=True, lr=self.BASE_LR)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer, factor=0.5, patience=5, verbose=True)
        self.stopper = EarlyStopper(patience=5, min_delta_percent=1.02)
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader

    
    def train(self):
        run = wandb.init(project="proyecto-recsys", config={
            "vocab_size": self.vocab_size,
            "embedding_dim": self.embedding_dim,
            "hidden_dim": self.hidden_dim,
            "sequence_length": self.sequence_length,
            "epochs": self.EPOCHS
        })

        self.model.to(device)

        for epoch in range(EPOCHS):    
            stopped = self._train_epoch(epoch)
            
            if stopped:
                print(f"Early stopping at epoch {epoch}")
                break

        run.finish()
        return self.model

    def _train_epoch(self, epoch):
        print(f"Training model on epoch {epoch}")
        i = 1
        losses = []
        for xs, ys in self.train_dataloader:
            self.optimizer.zero_grad()
            
            xs, ys = xs.to(device), ys.to(device)

            output = self.loss(self.model(xs), ys)
            output.backward()
            self.optimizer.step()

            losses.append(output.item())

            if i % self.PRINT_EVERY == 0:
                avg_loss = sum(losses) / len(losses)
                losses = []
                wandb.log({"train_loss": avg_loss, "epoch": epoch, "step": i})

            if i % self.VAL_EVERY == 0:
                print("\nEvaluating model on val set ...")
                self.scheduler.step()
                self.model.eval()

                with torch.no_grad():
                    val_iter = iter(self.val_dataloader)

                    val_losses = []            
                    for xs, ys in tqdm.tqdm(val_iter, total=len(val_iter)):
                        xs, ys = xs.to(device), ys.to(device)
                        output = self.loss(self.model(xs), ys)
                        val_losses.append(output.item())

                avg_val_loss = sum(val_losses) / len(val_losses)

                wandb.log({"val_loss": avg_val_loss, "epoch": epoch, "step": i, 
                            "lr" : self.optimizer.param_groups[0]['lr']})
                
                stop = self.stopper.early_stop(avg_val_loss)
                if stop: return True
                
                self.model.train()

            i += 1

### Parameters

In [ ]:
import itertools



emb_dims = [16, 32, 64, 128, 256]
hidden_dims = [10, 20, 40, 80, 150]

emb_dims = [100]
hidden_dims = [80]

for emb_dim, hidden_dim in itertools.product(emb_dims, hidden_dims):
    trainer = Trainer(vocab_size=len(pois), embedding_dim=emb_dim, hidden_dim=hidden_dim, sequence_length=SEQUENCE_LENGTH - 1, 
                    train_dataloader=train_dataloader, val_dataloader=val_dataloader)
    trainer.train()

AAA 73184 100 80 13


Training model on epoch 0
torch.Size([64, 13]) torch.Size([64])
tensor([[-11.1966, -11.0238, -11.2010,  ..., -11.4881, -11.6892, -11.1198],
        [-10.7591, -11.0268, -11.3280,  ..., -11.4441, -11.1429, -11.0685],
        [-11.1954, -11.0401, -11.1271,  ..., -11.3258, -11.8648, -11.1997],
        ...,
        [-10.8120, -11.2442, -10.8836,  ..., -11.2783, -11.1713, -11.3492],
        [-11.0726, -11.3178, -11.1701,  ..., -11.3965, -11.7300, -11.0785],
        [-11.0979, -10.9380, -11.0879,  ..., -11.8757, -11.5162, -11.0749]],
       grad_fn=<LogSoftmaxBackward0>)
torch.Size([64, 13]) torch.Size([64])
tensor([[-11.1411, -11.1864, -11.4283,  ..., -11.3508, -11.3988, -10.9607],
        [-11.2662, -11.1984, -11.4280,  ..., -11.5392, -11.5037, -11.6380],
        [-11.0455, -11.4308, -10.7337,  ..., -11.3897, -11.5293, -11.2884],
        ...,
        [-11.0266, -11.3090, -11.1635,  ..., -11.2155, -11.2710, -11.2368],
        [-11.0629, -10.9649, -11.2431,  ..., -11.0655, -11.4106, -11.1855

KeyboardInterrupt: 

wandb: Waiting for W&B process to finish... (success).


In [ ]:
print(EMBEDDING_DIM, HIDDEN_DIM)


100 80


In [ ]:

# Globals
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
loss = nn.NLLLoss()

# Training Parameters
print_every = 100
val_every = 1000 # 2000
adjust_every = 800 # 500
base_lr = 1e-2
epsilon_0 = 1e-2

# Get the dataset

# Create the model
print("Initializing model ...")
lm = EmbeddingModel(
    vocab_size=len(pois),
    emb_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    sample_length=SEQUENCE_LENGTH - 1
)


lm.to(device)
optimizer = optim.SGD(lm.parameters(), lr=epsilon_0) 
# optimizer = optim.Adam(lm.parameters(), amsgrad=True, lr=base_lr)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)


losses = []

print("Beginning training ...")
i = 0
for epoch in range(EPOCHS):    
    # Zero the parameter gradients
    print("Epoch", epoch + 1)
    for xs, ys in train_dataloader:
        optimizer.zero_grad()
        
        xs, ys = xs.to(device), ys.to(device)

        # Get the averaged batch loss
        output = loss(lm(xs), ys)

        # Run the backward pass (calculate gradients)
        output.backward()

        # Update the model
        optimizer.step()

        # Save and print statistics
        losses.append(output.item())

        if i % print_every == (print_every - 1):
            # Get average loss
            avg_loss = sum(losses) / len(losses)
            losses = []

            # Compute perplexity
            perp = math.exp(avg_loss)

            # Print and log the train loss
            print("({}) : {}".format((i + 1), perp))
            # writer.add_scalar('Training Loss', perp, i)

        if i % val_every == (val_every - 1):
            print("\nEvaluating model on val set ...")
            
            lm.eval()

            with torch.no_grad():
                val_iter = iter(val_dataloader)

                val_losses = []            
                for xs, ys in tqdm.tqdm(val_iter, total=len(val_iter)):
                    xs, ys = xs.to(device), ys.to(device)
                    output = loss(lm(xs), ys)
                    val_losses.append(output.item())


            avg_val_loss = sum(val_losses) / len(val_losses)
            val_losses = []

            # Compute perplexity
            val_perp = math.exp(avg_val_loss)

            # Print and log the val loss
            print("Average val Loss: {}\n".format(val_perp))
            # writer.add_scalar('val Loss', val_perp, i)
                    
            lm.train()

        # Update the learning rate every iteration, just as in the paper
        if i % adjust_every == (adjust_every - 1):
            for g in optimizer.param_groups:
                g['lr'] = epsilon_0 / (1 + r * (i / adjust_every))

        i += 1

Initializing model ...
AAA 73184 100 80 13
Beginning training ...
Epoch 1
(100) : 75516.47713167322


KeyboardInterrupt: 

In [ ]:
def accuracy(model, test_seqs, test_targets):
    model = model.to(device)
    batch_size = 64
    corrects = 0
    for index in range(0, len(test_seqs), batch_size):
        x = torch.tensor(test_seqs[idx:idx + batch_size]).to(device)
        y = torch.tensor(test_targets[idx:idx + batch_size]).to(device)
        pred = model.predict(x)
        corrects += torch.sum(pred == y).item()
    return  corrects / len(test_seqs)

In [ ]:
accuracy(lm, test_seq_history, test_seq_target)

0.0